<a href="https://colab.research.google.com/github/alinakhodotovych2022/Project2-housing-price-ml/blob/main/Project2_housing_price_ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Housing Price Prediction Using Zillow ZHVI (ZIP-Level)

_Data Science Project — Machine Learning Modeling & Pipeline (Project 2)_

<p align="center">
  <img src="./image Project 2.jpeg" alt="Housing Price Banner" width="700">
</p>




## 1. Project Overview (BLUF)

**Bottom Line Up Front:**  
This project builds a **reproducible machine learning regression pipeline** that predicts home values (Zillow Home Value Index, ZHVI) at the **ZIP code level** using historical ZHVI time series data from **Zillow Research**.

The pipeline includes:

- Data loading and cleaning of the wide monthly ZHVI dataset  
- Reshaping from wide format (many date columns) to modeling-ready features  
- Exploratory Data Analysis (EDA) with clear visualizations  
- Feature engineering on recent monthly ZHVI history  
- Baseline and advanced regression models (e.g., Linear Regression, Ridge, Random Forest)  
- **Scikit-learn `Pipeline`** with preprocessing, feature engineering, and model in one object  
- **Hyperparameter tuning** with cross-validation  
- Final model evaluation using MAE, RMSE, and R²  
- **Serialization of the trained pipeline** for reuse (`housing_price_pipeline.pkl`)

Business question in simple terms:

> _“Given recent price dynamics for each ZIP code, what is a reasonable prediction of the current home value (ZHVI)?”_


## 2. Setup and Data Loading

In [ ]:
# STEP 0: Import core libraries

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor

import joblib

# Configure plots to be a bit larger by default
plt.rcParams['figure.figsize'] = (8, 5)

### 2.1 Load Zillow ZHVI ZIP-level dataset

In Colab we can upload the CSV directly.  
If you run this locally, you can skip the `files.upload()` part and read the CSV from a file path instead.


In [ ]:
# STEP 1: Load the ZHVI ZIP-level CSV file
# Option A (Google Colab upload):
# --------------------------------
try:
    from google.colab import files  # Will fail if not in Colab
    uploaded = files.upload()
    csv_filename = list(uploaded.keys())[0]
except Exception:
    csv_filename = "Zip_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv"  # adjust if needed

# Read the dataset
df = pd.read_csv(csv_filename)

# Quick checks
df.head()

### 2.2 Initial Data Inspection

In [ ]:
# STEP 2: Basic dataset structure and summary

print("Shape (rows, columns):", df.shape)
print("\nInfo:")
print(df.info())

print("\nFirst 5 rows:")
display(df.head())

## 3. Data Preparation & Reshaping

The Zillow ZIP-level file is in **wide format**: each row is a ZIP code and each date (YYYY-MM-DD) is a separate column.  
To build features, we first need to:

1. Identify which columns are **date columns**.  
2. Sort those date columns in chronological order.  
3. Decide which **latest month** we want to predict (our target).  
4. Construct features from the recent history (e.g., last 12 months).

In [ ]:
# STEP 3: Identify date columns (monthly ZHVI values)

# Non-date columns that describe the region
meta_cols = [
    "RegionID", "SizeRank", "RegionName", "RegionType",
    "StateName", "State", "City", "Metro", "CountyName"
]

# Treat all other columns as candidate date columns
candidate_date_cols = [c for c in df.columns if c not in meta_cols]

# Many of these columns are date-like strings such as '2000-01-31'
# We keep only those that can be safely parsed as dates
date_cols = []
for c in candidate_date_cols:
    try:
        pd.to_datetime(c)
        date_cols.append(c)
    except Exception:
        # skip non-date columns if any
        pass

# Sort the date columns chronologically
date_cols = sorted(date_cols, key=lambda x: pd.to_datetime(x))

print(f"Number of monthly ZHVI columns: {len(date_cols)}")
print("First 5 date columns:", date_cols[:5])
print("Last 5 date columns:", date_cols[-5:])

### 3.1 Define target month and feature window

We will:

- Use the **latest available month** as the regression target `y`.  
- Use the **12 months immediately before the target** as raw numeric features.  
- Later, we will also derive **growth-rate features** from this 12‑month history.


In [ ]:
# STEP 4: Define target month and feature window

# Use the last available month as the prediction target
target_col = date_cols[-1]
print("Target month column:", target_col)

# Choose a 12‑month window before the target as base numeric features
window_months = 12
feature_date_cols = date_cols[-(window_months + 1):-1]  # previous 12 months
print("\nFeature window columns (12 months before target):")
print(feature_date_cols)

### 3.2 Build base modeling dataset

We keep only rows where both the target month and the 12-month window are non-null.  
This ensures the model trains on complete histories.


In [ ]:
# STEP 5: Filter rows with complete data for the chosen window + target

cols_needed = meta_cols + feature_date_cols + [target_col]
model_df = df[cols_needed].dropna()

print("Shape after dropping missing rows:", model_df.shape)

# Rename target for convenience
model_df = model_df.rename(columns={target_col: "target_zhvi"})

## 4. Feature Engineering

From the 12‑month ZHVI history we generate additional features, for example:

- Average ZHVI over the last 12 months  
- Standard deviation (volatility)  
- 12‑month growth rate  
- 3‑month and 6‑month growth rates  
- Relative position of the latest month vs. 12‑month mean  

We also keep region descriptors such as **State** and **City** as categorical features.


In [ ]:
# STEP 6: Create numeric time‑series features from the 12‑month window

feature_df = model_df.copy()

# Convert the feature window into a numpy array for calculations
window_values = feature_df[feature_date_cols].values

# Basic statistics
feature_df["zhvi_mean_12m"] = window_values.mean(axis=1)
feature_df["zhvi_std_12m"] = window_values.std(axis=1)

# Last month in the window (just before target)
last_window_col = feature_date_cols[-1]
feature_df["zhvi_last_before_target"] = feature_df[last_window_col]

# Growth features
first_window_col = feature_date_cols[0]
feature_df["growth_12m_abs"] = feature_df[last_window_col] - feature_df[first_window_col]
feature_df["growth_12m_pct"] = feature_df["growth_12m_abs"] / feature_df[first_window_col]

# 6‑month growth (if at least 6 months in window)
if len(feature_date_cols) >= 6:
    col_6m_ago = feature_date_cols[-6]
    feature_df["growth_6m_abs"] = feature_df[last_window_col] - feature_df[col_6m_ago]
    feature_df["growth_6m_pct"] = feature_df["growth_6m_abs"] / feature_df[col_6m_ago]

# 3‑month growth (if at least 3 months in window)
if len(feature_date_cols) >= 3:
    col_3m_ago = feature_date_cols[-3]
    feature_df["growth_3m_abs"] = feature_df[last_window_col] - feature_df[col_3m_ago]
    feature_df["growth_3m_pct"] = feature_df["growth_3m_abs"] / feature_df[col_3m_ago]

# Relative position vs. mean
feature_df["last_vs_mean_ratio"] = feature_df[last_window_col] / feature_df["zhvi_mean_12m"]

print("Feature dataframe shape (with engineered features):", feature_df.shape)
feature_df.head()

### 4.1 Final feature set

We select:

- **Numeric features**: engineered statistics and growth rates  
- **Categorical features**: `State` and `Metro`  
- **Target**: `target_zhvi`


In [ ]:
# STEP 7: Define feature matrix X and target vector y

numeric_features = [
    "zhvi_mean_12m", "zhvi_std_12m",
    "zhvi_last_before_target",
    "growth_12m_abs", "growth_12m_pct",
    "growth_6m_abs", "growth_6m_pct",
    "growth_3m_abs", "growth_3m_pct",
    "last_vs_mean_ratio"
]

categorical_features = ["State", "Metro"]

# X = selected features; y = target
X = feature_df[numeric_features + categorical_features]
y = feature_df["target_zhvi"]

print("X shape:", X.shape)
print("y shape:", y.shape)

## 5. Exploratory Data Analysis (EDA)

We now explore the data to understand:

- Distribution of the target (current ZHVI)  
- Relationship between engineered features and target  
- State-level patterns  


In [ ]:
# STEP 8: Target distribution

plt.figure()
plt.hist(y, bins=40)
plt.title("Distribution of Target ZHVI (Latest Month)")
plt.xlabel("ZHVI")
plt.ylabel("Count")
plt.show()

In [ ]:
# STEP 9: Example relationship — 12‑month growth vs target

plt.figure()
plt.scatter(feature_df["growth_12m_pct"], y, alpha=0.3)
plt.title("12‑Month Growth Rate vs Target ZHVI")
plt.xlabel("12‑Month Growth Rate (pct)")
plt.ylabel("Target ZHVI")
plt.show()

In [ ]:
# STEP 10: Average target ZHVI by State (top 10 states by mean)

state_means = feature_df.groupby("State")["target_zhvi"].mean().sort_values(ascending=False).head(10)
print(state_means)

plt.figure()
state_means.plot(kind="bar")
plt.title("Top 10 States by Average Target ZHVI")
plt.ylabel("Average ZHVI")
plt.xticks(rotation=45)
plt.show()

## 6. Train/Test Split

We split the data into **training** and **test** sets using a random split, since each row corresponds to a different ZIP code (not a time-series split per ZIP).

In [ ]:
# STEP 11: Train/test split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("Train shape:", X_train.shape)
print("Test shape:", X_test.shape)

## 7. Baseline Model and Preprocessing Pipeline

Before advanced models, we create:

1. A **baseline model** using `DummyRegressor` (predicts median).  
2. A **preprocessing pipeline** with:
   - `StandardScaler` for numeric features  
   - `OneHotEncoder` for categorical features  


In [ ]:
# STEP 12: Baseline model (DummyRegressor)

baseline = DummyRegressor(strategy="median")
baseline.fit(X_train, y_train)
baseline_preds = baseline.predict(X_test)

baseline_mae = mean_absolute_error(y_test, baseline_preds)
baseline_rmse = mean_squared_error(y_test, baseline_preds, squared=False)
baseline_r2 = r2_score(y_test, baseline_preds)

print("Baseline MAE:", baseline_mae)
print("Baseline RMSE:", baseline_rmse)
print("Baseline R^2:", baseline_r2)

In [ ]:
# STEP 13: Build preprocessing transformer for numeric + categorical features

numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown="ignore")

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

preprocessor

## 8. Modeling Strategy

We evaluate several regression algorithms using the same preprocessing:

- **Linear Regression** (baseline ML model)  
- **Ridge Regression** (L2-regularized linear model)  
- **Random Forest Regressor** (non-linear tree ensemble)

We use cross-validation on the training set to compare models, then perform hyperparameter tuning on the best ones.


In [ ]:
# STEP 14: Helper function for evaluation

def evaluate_model(model, X_train, y_train, X_test, y_test, model_name="model"):
    """Fit model, compute CV score, and evaluate on test set."""
    # Cross-validation on training set
    cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring="neg_mean_squared_error")
    cv_rmse = np.mean(np.sqrt(-cv_scores))

    # Fit on full training data
    model.fit(X_train, y_train)
    preds = model.predict(X_test)

    mae = mean_absolute_error(y_test, preds)
    rmse = mean_squared_error(y_test, preds, squared=False)
    r2 = r2_score(y_test, preds)

    print(f"\n=== {model_name} ===")
    print("CV RMSE (mean over folds):", cv_rmse)
    print("Test MAE:", mae)
    print("Test RMSE:", rmse)
    print("Test R^2:", r2)

    return {
        "model_name": model_name,
        "cv_rmse": cv_rmse,
        "test_mae": mae,
        "test_rmse": rmse,
        "test_r2": r2,
    }

In [ ]:
# STEP 15: Define models wrapped in Pipelines with shared preprocessing

linreg_model = Pipeline(steps=[
    ("preprocess", preprocessor),
    ("model", LinearRegression())
])

ridge_model = Pipeline(steps=[
    ("preprocess", preprocessor),
    ("model", Ridge())
])

rf_model = Pipeline(steps=[
    ("preprocess", preprocessor),
    ("model", RandomForestRegressor(random_state=42))
])

results = []
results.append(evaluate_model(linreg_model, X_train, y_train, X_test, y_test, "Linear Regression"))
results.append(evaluate_model(ridge_model, X_train, y_train, X_test, y_test, "Ridge Regression"))
results.append(evaluate_model(rf_model, X_train, y_train, X_test, y_test, "Random Forest"))

results_df = pd.DataFrame(results)
display(results_df)

## 9. Hyperparameter Tuning with GridSearchCV

Based on initial results, we tune **Ridge Regression** using a grid of `alpha` values.  
This step demonstrates how to integrate preprocessing + model selection into a single `GridSearchCV` pipeline.


In [ ]:
# STEP 16: Hyperparameter tuning for Ridge Regression

ridge_pipeline = Pipeline(steps=[
    ("preprocess", preprocessor),
    ("model", Ridge())
])

param_grid = {
    "model__alpha": [0.1, 1, 10, 50, 100]
}

grid = GridSearchCV(
    ridge_pipeline,
    param_grid=param_grid,
    cv=5,
    scoring="neg_mean_squared_error",
    n_jobs=-1
)

grid.fit(X_train, y_train)

print("Best parameters:", grid.best_params_)
print("Best CV score (negative MSE):", grid.best_score_)

In [ ]:
# STEP 17: Final evaluation of tuned Ridge model

best_model = grid.best_estimator_
final_preds = best_model.predict(X_test)

final_mae = mean_absolute_error(y_test, final_preds)
final_rmse = mean_squared_error(y_test, final_preds, squared=False)
final_r2 = r2_score(y_test, final_preds)

print("Final tuned Ridge MAE:", final_mae)
print("Final tuned Ridge RMSE:", final_rmse)
print("Final tuned Ridge R^2:", final_r2)

## 10. Model Diagnostics & Visualization

In [ ]:
# STEP 18: Actual vs Predicted scatter plot

plt.figure()
plt.scatter(y_test, final_preds, alpha=0.4)
plt.title("Actual vs Predicted ZHVI (Tuned Ridge Model)")
plt.xlabel("Actual ZHVI")
plt.ylabel("Predicted ZHVI")
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], "k--")  # diagonal line
plt.show()

In [ ]:
# STEP 19: Residuals histogram

residuals = y_test - final_preds

plt.figure()
plt.hist(residuals, bins=40)
plt.title("Residual Distribution (Actual - Predicted)")
plt.xlabel("Residual")
plt.ylabel("Count")
plt.show()

In [ ]:
# STEP 20: Approximate feature importance for numeric features (Ridge coefficients)

# Extract the trained Ridge model from the pipeline
ridge_step = best_model.named_steps["model"]

# Get one‑hot encoder output feature names for categorical variables
ohe = best_model.named_steps["preprocess"].named_transformers_["cat"]
cat_feature_names = ohe.get_feature_names_out(categorical_features)

all_feature_names = numeric_features + list(cat_feature_names)

# Coefficients aligned with transformed feature space
coeffs = ridge_step.coef_

importance_df = pd.DataFrame({
    "feature": all_feature_names,
    "coefficient": coeffs
})

# Sort by absolute coefficient value and show top 15
importance_df["abs_coef"] = importance_df["coefficient"].abs()
top_importance = importance_df.sort_values("abs_coef", ascending=False).head(15)

display(top_importance)

plt.figure()
plt.barh(top_importance["feature"], top_importance["coefficient"])
plt.title("Top 15 Features by Ridge Coefficient")
plt.xlabel("Coefficient")
plt.gca().invert_yaxis()
plt.show()

## 11. Save Trained Pipeline

We serialize the final tuned pipeline using `joblib`.  
This file can be loaded later to generate predictions on new data without retraining.


In [ ]:
# STEP 21: Save the trained pipeline to disk

pipeline_filename = "housing_price_pipeline.pkl"
joblib.dump(best_model, pipeline_filename)

print(f"Saved trained pipeline to: {pipeline_filename}")

## 12. Business Interpretation & Next Steps

### 12.1 Interpretation of Results

- The tuned Ridge model achieves **lower error metrics than the baseline** median regressor, indicating that recent 12‑month ZHVI history contains useful predictive signal.  
- Growth-related features (12‑month and 6‑month percentage changes) and recent level (`zhvi_last_before_target`) typically have the strongest coefficients, which aligns with intuition about housing markets.  
- State- and metro-level one‑hot encoded features capture regional pricing differences that are not explained by recent dynamics alone.

### 12.2 Example Use Cases

Stakeholders who might benefit from this model:

- **Real estate investors**: to compare ZIP codes and identify areas with strong momentum but still moderate prices.  
- **Lenders / banks**: to stress‑test portfolios under different regional value assumptions.  
- **Policy analysts or city planners**: to understand how local markets differ in level and growth.

### 12.3 Limitations

- The model uses only **historical ZHVI data** — it does not include macroeconomic variables (rates, income, employment, etc.).  
- We treat each ZIP as independent and perform a random train/test split, which does not fully respect temporal dynamics at the national level.  
- Out‑of‑sample performance may degrade if market conditions change dramatically relative to the training period.

### 12.4 Possible Extensions

- Add **macro features** (unemployment rate, mortgage rates, income, construction permits).  
- Use **time‑series models** (e.g., gradient boosting with lag features or dedicated sequence models).  
- Tune and compare additional algorithms (e.g., Gradient Boosting, XGBoost, LightGBM) and perform deeper hyperparameter searches.
